# Notebook 04 — Modelagem de Clusterização de Criptomoedas
Neste notebook aplicamos técnicas de aprendizado não supervisionado para agrupar criptomoedas com base em suas características de risco e comportamento.

**Etapas principais:**
- Carregar features processadas
- Normalizar os dados
- Treinar K-Means (k=5)
- Avaliar clusters com Silhouette Score
- Visualizar clusters via PCA (2D)
- Analisar perfis médios dos clusters
- Salvar resultado final para uso posterior


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")

features = pd.read_csv("../data/processed/crypto_features.csv", index_col="coin")
features.head()

## Preparação dos Dados para Modelagem
Seleciona as variáveis relevantes, aplica normalização e prepara os dados para o algoritmo de clusterização.

In [ ]:
# Seleção das features corretas
X_model = features[['avg_return','volatility','downside_std',
                    'abs_return','log_volume','log_mc']].copy()

# Aplicar peso ao marketcap (log_mc)
X_model['log_mc'] = X_model['log_mc'] * 5 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_model)

## Aplicando K-Means para Agrupamento
Executa o algoritmo K-Means para identificar grupos de criptomoedas com perfis de risco semelhantes.

In [ ]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

features['cluster'] = labels
features.head()

## Avaliação do Modelo — Silhouette Score
Calcula o Silhouette Score para avaliar a qualidade da separação entre os clusters formados.

In [ ]:
sil = silhouette_score(X_scaled, labels)
print("Silhouette Score:", sil)

## Redução de Dimensionalidade com PCA
Utiliza PCA para reduzir as dimensões dos dados e permitir a visualização dos clusters em 2D.

In [ ]:
pca = PCA(n_components=2)
coords = pca.fit_transform(X_scaled)

features['pc1'] = coords[:,0]
features['pc2'] = coords[:,1]

## Visualização dos Clusters em 2D
Gera um gráfico de dispersão dos clusters identificados, facilitando a interpretação visual dos grupos formados.

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(
    x=features['pc1'],
    y=features['pc2'],
    hue=features['cluster'],
    palette='Set1',
    s=130
)

for coin, row in features.iterrows():
    plt.text(row['pc1']+0.02, row['pc2']+0.02, coin, fontsize=9)

plt.title("Clusters de Cripto — PCA 2D")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

## Análise dos Perfis Médios de Cada Cluster
Calcula e apresenta as médias das principais métricas para cada grupo, permitindo entender as características de risco e retorno dos clusters formados.

In [ ]:
cluster_profile = features.groupby('cluster').mean()[[
    'avg_return','volatility','downside_std','abs_return',
    'avg_volume','avg_marketcap'
]]

cluster_profile

## Visualização dos Perfis de Cluster (Heatmap)
Gera um heatmap para comparar visualmente as médias das métricas entre os diferentes clusters.

In [ ]:
plt.figure(figsize=(9,5))
sns.heatmap(cluster_profile, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Perfil Médio de Cada Cluster")
plt.show()

## Salvamento dos Resultados de Clusterização
Exporta os resultados finais dos clusters para arquivos CSV, incluindo a versão com nomes de grupos para facilitar a análise e apresentação.

In [ ]:
output = "../data/outputs/crypto_clusters.csv"
features.to_csv(output)

print("Clusters salvos em:", output)
features

## Teste

Não considerar

In [ ]:
cluster_profile = features.groupby('cluster').agg({
    'avg_marketcap': 'mean',
    'volatility': 'mean',
    'avg_return': 'mean'
}).reset_index()


cluster_profile = cluster_profile.sort_values(by='avg_marketcap', ascending=False)
cluster_profile['cluster_label'] = [
    'Blue Chip Estável',
    'Altcoin Sólida',
    'Altcoin Intermediária',
    'Geminha (Projeto Pequeno Promissor)',
    'Altcoin Volátil'
]

cluster_map = dict(zip(cluster_profile['cluster'], cluster_profile['cluster_label']))

features['cluster_label'] = features['cluster'].map(cluster_map)

output = "../data/outputs/crypto_clusters_renomeados.csv"
features.to_csv(output)